In [1]:
### Example of many vectors connected to a single copy node ###

import numpy as np
from contractn import TN
tn = TN()

# Add central copy tensor of order 101
copy_node = tn.add_copy_node(101)

# Connect vectors to all but one edge of the copy tensor
for i in range(100):
    vec = np.array([1, 0.99])
    vec_node = tn.add_dense_node(vec)
    # Connect i'th axis of copy_node to 0'th axis of vec_node
    tn.connect_nodes(copy_node, vec_node, i, 0)

print(tn.contract())  # array([1., 0.36603234])
%time tn.contract()   # 6.85 ms

[1.         0.36603234]
CPU times: user 7.13 ms, sys: 0 ns, total: 7.13 ms
Wall time: 6.99 ms


array([1.        , 0.36603234])

In [2]:
### Example of Tucker vs. CP decomposition and einsum ###

import numpy as np
from contractn import TN

# Initialize TNs for 3rd-order Tucker and CP decompositions
cp = TN()
tucker = TN()

# Add central "hub" cores
cp_hub = cp.add_copy_node(3)
tucker_hub = tucker.add_dense_node(np.ones((4, 4, 4)))

# Connect each hub to three factor matrices
for i in range(3):
    mat = np.eye(4, 10)
    cp_mat = cp.add_dense_node(mat)
    tucker_mat = tucker.add_dense_node(mat)
    cp.connect_nodes(cp_hub, cp_mat, i, 0)
    tucker.connect_nodes(tucker_hub, tucker_mat, i, 0)

print(cp.einsum_str)      # "ac,ad,ae->cde"
print(tucker.einsum_str)  # "abc,ae,bf,cg->efg"

ac,ad,ae->cde
abc,ae,bf,cg->efg


In [3]:
### Example of multiplying many matrices ###

import numpy as np
from contractn import TN

# Initialize and connect together vector
# to a chain of 1000 3x3 matrices
tn = TN()
prev_node = tn.add_dense_node(np.ones((3,)))
for i in range(1000):
    mat_node = tn.add_dense_node(np.ones((3, 3)))
    tn.connect_nodes(prev_node, mat_node, -1, 0)
    prev_node = mat_node

print(tn.contract())
    # [inf inf inf]
print(tn.contract(split_format=True))
    # (array([1., 1., 1.]), array(1098.61228867))

[inf inf inf]
(array([1., 1., 1.]), array(1098.61228867))


/home/jemis/ContracTN/contractn/einsum.py:75: RuntimeWarning: overflow encountered in exp
  return get_func("exp", backend)(x)
